# Kickstarter ML Project

## Preparation

In [4]:
import pandas as pd
import numpy as np
import glob

import sweetviz as sv

ModuleNotFoundError: No module named 'sweetviz'

### Load Data

In [2]:
# Load csv's and merge to a single dataframe

path = "/Users/bur.oez/neuefische/Kickstarter-ML-Project/data/*.csv" # Burak's path
#path = "data/*.csv" # Christian's path
#path = ".../*.csv"" # Matthias's path

all_files = glob.glob(path)
df_raw = pd.concat((pd.read_csv(f) for f in all_files))

### Overview

In [4]:
## Execute Sweetviz 
#my_report = sv.analyze(df_raw)
#my_report.show_html() # Default arguments will generate to "SWEETVIZ_REPORT.html"

Done! Use 'show' commands to display/save.   |██████████| [100%]   00:03 -> (00:00 left)
Report SWEETVIZ_REPORT.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [3]:
df_raw.columns

Index(['backers_count', 'blurb', 'category', 'converted_pledged_amount',
       'country', 'created_at', 'creator', 'currency', 'currency_symbol',
       'currency_trailing_code', 'current_currency', 'deadline',
       'disable_communication', 'friends', 'fx_rate', 'goal', 'id',
       'is_backing', 'is_starrable', 'is_starred', 'launched_at', 'location',
       'name', 'permissions', 'photo', 'pledged', 'profile', 'slug',
       'source_url', 'spotlight', 'staff_pick', 'state', 'state_changed_at',
       'static_usd_rate', 'urls', 'usd_pledged', 'usd_type'],
      dtype='object')

## Data Cleaning

For safety, we operate on a copy of the data frame.

In [202]:
df = df_raw.copy()

For a basic feature analysis, we may focus on easily accessible features. Consequently, we drop the cols
- `blurb`, `creator`, `slug`, `name` and `photo`, because they might only be exploitable through a semantic or context analysis,
- `currency_symbol`, `currency_trailing_code`, because they are redundant,
r a basic feature analysis, we may focus on easily accessible features. Consequently, we drop the cols
- `blurb`, `creator`, `slug`, `name` and `photo`, because they might only be exploitable through a semantic or context analysis,
- `currency_symbol`, `currency_trailing_code`, because they are redundant,
- `friends`, ìs_starrable` and `permission`, because they do not contain any information,
- `disable_communication`, `is_backing` and `is_starred`, because they have an entry only for the same 300 data points and it is questionable whether the missing entries may be treated as one category; later on we might try this with one of them and drop the other two,  
- `urls`, `source_url` and `profile` because they do not contain additional information.
- `spotlight` and `staff_pick` will be dropped, because the creator has no influence on Kickstarter Staff picks
- `created_at`, `launched_at` and `state_changed_at` will be kept to calculate time-deltas for potential new features.

In [203]:
# dropping columns
df = df.drop(["blurb", "creator", "slug", "name", "photo", "currency_symbol", "currency_trailing_code", "friends", "permissions", "disable_communication", "is_backing", "is_starred","urls", "source_url", "profile", "usd_type", "spotlight", "staff_pick","is_starrable"], axis = 1)
df.head()

,backers_count,category,converted_pledged_amount,country,created_at,currency,current_currency,deadline,fx_rate,goal,id,launched_at,location,pledged,state,state_changed_at,static_usd_rate,usd_pledged
0,315,"{""id"":266,""name"":""Footwear"",""slug"":""fashion/fo...",28645,US,1541459205,USD,USD,1552539775,1.000000,28000.0,2108505034,1548223375,"{""id"":2462429,""name"":""Novato"",""slug"":""novato-c...",28645.0,live,1548223375,1.000000,28645.000000
1,47,"{""id"":273,""name"":""Playing Cards"",""slug"":""games...",1950,US,1501684093,USD,USD,1504976459,1.000000,1000.0,928751314,1502384459,"{""id"":2400549,""name"":""Euless"",""slug"":""euless-t...",1950.0,successful,1504976459,1.000000,1950.000000
2,271,"{""id"":43,""name"":""Rock"",""slug"":""music/rock"",""po...",22404,US,1348987533,USD,USD,1371013395,1.000000,15000.0,928014092,1368421395,"{""id"":2423474,""name"":""Hollywood"",""slug"":""holly...",22404.0,successful,1371013395,1.000000,22404.000000
3,3,"{""id"":273,""name"":""Playing Cards"",""slug"":""games...",165,GB,1483780271,GBP,USD,1489425776,1.308394,10000.0,596091328,1484245376,"{""id"":475457,""name"":""Kaunas"",""slug"":""kaunas-ka...",136.0,failed,1489425776,1.216066,165.384934
4,3,"{""id"":48,""name"":""Nonfiction"",""slug"":""publishin...",2820,US,1354817071,USD,USD,1357763527,1.000000,2800.0,998516049,1355171527,"{""id"":2507703,""name"":""Traverse City"",""slug"":""t...",2820.0,successful,1357763527,1.000000,2820.000000


Extract field and subfield ids from `category` (id and parent_id):

In [204]:
import ast
df.category =[ ast.literal_eval(x) for x in df.iloc[:,df.columns.tolist().index("category")]]

In [205]:
# checking which rows do not contain "parent_id" inside category dictionary
cat_list_parent_id = np.array(["parent_id" not in df.category.iloc[x].keys() for x in range(0,len(df.category))])
cat_list_parent_id.sum()

9041

In [207]:
df[cat_list_parent_id].iloc[0:3]

,backers_count,category,converted_pledged_amount,country,created_at,currency,current_currency,deadline,fx_rate,goal,id,launched_at,location,pledged,state,state_changed_at,static_usd_rate,usd_pledged
7,33,"{'id': 14, 'name': 'Music', 'slug': 'music', '...",660,US,1546965483,USD,USD,1550067305,1.000000,400.0,1481360049,1547475305,"{""id"":2430903,""name"":""Kaysville"",""slug"":""kaysv...",660.0,successful,1550067307,1.000000,660.000000
25,74,"{'id': 15, 'name': 'Photography', 'slug': 'pho...",2808,GB,1542202969,GBP,USD,1544200381,1.308394,2000.0,865110410,1542385981,"{""id"":12056,""name"":""Bath"",""slug"":""bath-gb"",""sh...",2199.0,successful,1544200381,1.301163,2861.258251
27,41,"{'id': 9, 'name': 'Fashion', 'slug': 'fashion'...",21161,GB,1551345787,GBP,CAD,1554490790,1.748586,20000.0,1889386358,1551898790,"{""id"":40611,""name"":""Winchester"",""slug"":""winche...",12102.0,live,1551898791,1.315996,15926.177904


In [208]:
df.category.iloc[0]

{'id': 266,
 'name': 'Footwear',
 'slug': 'fashion/footwear',
 'position': 5,
 'parent_id': 9,
 'color': 16752598,
 'urls': {'web': {'discover': 'http://www.kickstarter.com/discover/categories/fashion/footwear'}}}

The `id` column represents each subcategory. `parent_id` represents each parent category, since the values for `id` start after `parent_id` values, we will use `id` to seperate each project into categories.

e `id` column represents each subcategory. `parent_id` represents each parent category, since the values for `id` start after `parent_id` values, we will use `id` to seperate each project into categories.

To projects without proper subcategorisation the `parent_id` is assigned. In addition `slug`is added for increased readability.

In [210]:
df["cat_id"] = [df.category.iloc[x]["id"] for x in range(0,len(df.category))]
df["slug"] = [df.category.iloc[x]["slug"] for x in range(0,len(df.category))]
categories = df.pop("category")

In [211]:
df.head(3)
print(df.shape)

(209222, 19)


In [ ]:
# check if currency - USD conversion with fx_rate and static_usd_rate matches usd_pledged value
print("pledged * static_usd_rate == usd_pledged")
print(df.eval("pledged * static_usd_rate == usd_pledged").value_counts())
print("pledged * fx_rate == usd_pledged")
print(df.eval("pledged * fx_rate == usd_pledged").value_counts())
print("converted_pledged_amount == usd_pledged")
print(df.eval("converted_pledged_amount == usd_pledged").value_counts())
print("pledged * static_usd_rate == converted_pledged_amount")
print(df.eval("pledged * static_usd_rate == converted_pledged_amount").value_counts())
print("pledged * fx_rate == converted_pledged_amount")
print(df.eval("pledged * fx_rate == converted_pledged_amount").value_counts())

In [ ]:
print(df.eval("converted_pledged_amount == pledged + usd_pledged").value_counts())

The values in `usd_pledged` do not match with the conversion of `pledged` using `fx_rate` or `static_usd_rate`. 

We decided to take the max value for these columns as the correct amount.

In [212]:
# checking whether pledged, usd_pledged or converted_pledged_amount has the highest values
df["max_pledged"] = [max(df.pledged.iloc[x], 
df.usd_pledged.iloc[x], 
df.converted_pledged_amount.iloc[x]) for x in range(0,len(df))]
df = df.drop(["usd_pledged", "pledged", "converted_pledged_amount"], axis = 1)

In [213]:
# in addition all other currency info can also be dropped
df = df.drop(["static_usd_rate", "fx_rate", "currency", "current_currency"], axis = 1)

In [214]:
# set checkpoint
df2 = df.copy()
df.head(2)
print(df.shape)

(209222, 13)


Now we will check whether `live` projects have met their goal and can be considered as `succesful`


In [215]:
df.loc[df['max_pledged'] >= df.goal, 'state'] = "successful"
#check whether there are live:successsful projects left
df.query("state == 'live' and max_pledged >= goal").state

Series([], Name: state, dtype: object)

In [249]:
df = df2.copy()
df.shape

(209222, 13)

Now we will drop all entries, which are not `successful` or `failed`


In [233]:
df.state.value_counts()

successful    117465
failed         75199
canceled        8624
live            7311
suspended        623
Name: state, dtype: int64

In [252]:
df = df[df.state != "canceled"]
df = df[df.state != "suspended"]
df = df[df.state != "live"]
df.state.unique()

array(['successful', 'failed'], dtype=object)

In [253]:
df.shape

(192664, 13)

Sort `countries` into categories US, NA, SEA, GB, ANZ, JP, EU:

In [254]:
pd.unique(df.country)

array(['US', 'GB', 'FR', 'AU', 'NZ', 'ES', 'IT', 'NO', 'NL', 'CA', 'SG',
       'MX', 'SE', 'IE', 'DE', 'BE', 'HK', 'AT', 'JP', 'DK', 'CH', 'LU'],
      dtype=object)

In [255]:
df.country.replace({'HK':"SEA", 'NL':"EU", 'AU':"ANZ", 'DE':"EU", "CA":"NA", 'SE':"EU",
                    'BE':"EU", 'MX':"NA", 'CH':"EU", 'SG':"SEA", 'FR':"EU", 'IT':"EU",
                    'DK':"EU", 'LU':"EU", 'NO':"EU", 'ES':"EU", 'IE':"EU", 'NZ':"ANZ", 
                    'AT':"EU"}, inplace = True)

Turn entries of `created_at`, `launched`, `deadline` into datetime:

In [256]:
df.created_at = pd.to_datetime(df.iloc[:,df.columns.tolist().index("created_at")],unit='s')
df.launched_at = pd.to_datetime(df.iloc[:,df.columns.tolist().index("launched_at")],unit='s')
df.deadline = pd.to_datetime(df.iloc[:,df.columns.tolist().index("deadline")],unit='s')
df.state_changed_at = pd.to_datetime(df.iloc[:,df.columns.tolist().index("state_changed_at")],unit='s')
df.head()

,backers_count,country,created_at,deadline,goal,id,launched_at,location,state,state_changed_at,cat_id,slug,max_pledged
1,47,US,2017-08-02 14:28:13,2017-09-09 17:00:59,1000.0,928751314,2017-08-10 17:00:59,"{""id"":2400549,""name"":""Euless"",""slug"":""euless-t...",successful,2017-09-09 17:00:59,273,games/playing cards,1950.000000
2,271,US,2012-09-30 06:45:33,2013-06-12 05:03:15,15000.0,928014092,2013-05-13 05:03:15,"{""id"":2423474,""name"":""Hollywood"",""slug"":""holly...",successful,2013-06-12 05:03:15,43,music/rock,22404.000000
3,3,GB,2017-01-07 09:11:11,2017-03-13 17:22:56,10000.0,596091328,2017-01-12 18:22:56,"{""id"":475457,""name"":""Kaunas"",""slug"":""kaunas-ka...",failed,2017-03-13 17:22:56,273,games/playing cards,165.384934
4,3,US,2012-12-06 18:04:31,2013-01-09 20:32:07,2800.0,998516049,2012-12-10 20:32:07,"{""id"":2507703,""name"":""Traverse City"",""slug"":""t...",successful,2013-01-09 20:32:07,48,publishing/nonfiction,2820.000000
5,35,US,2014-10-24 17:35:50,2015-05-02 02:25:46,3500.0,1224600291,2015-04-02 02:25:46,"{""id"":2354877,""name"":""Annapolis"",""slug"":""annap...",successful,2015-05-02 02:25:46,36,music/classical music,3725.000000


In [257]:
df.shape

(192664, 13)

In [258]:
# set checkpoint
df2 = df.copy()

Check for `id` copies and remove if they refer to the same data point: multiplicities of entries:

In [259]:
print(*df.id.value_counts().unique())

2 1


List of multiply used `id` entries:

In [260]:
dic = df.id.value_counts()
multiples = np.array([key for key in dic.keys() if dic[key] > 1])
len(multiples)

23685

They don't differ in any features, except one.

In [261]:
for id in multiples:
    print((id, [c for c in df.columns if df.query('id == '+str(id))[c].nunique() != 1]))


(484859009, [])
(378920150, [])
(825148597, [])
(940105647, [])
(708119476, [])
(1909365226, [])
(944999464, [])
(1869171908, [])
(1178203051, [])
(1833998747, [])
(59894696, [])
(886682531, [])
(351380385, [])
(64848610, [])
(2031038417, [])
(2023254936, [])
(1258618795, [])
(1803035537, [])
(844476521, [])
(316208677, [])
(1593603299, [])
(515416961, [])
(1536912248, [])
(146217846, [])
(57353073, [])
(1175792606, [])
(852282178, [])
(1425509220, [])
(1296676559, [])
(154363137, [])
(1268960436, [])
(9005823, [])
(508197722, [])
(1073017867, [])
(1940431686, [])
(789481316, [])
(1229889383, [])
(1681253417, [])
(2048321384, [])
(1370604629, [])
(1128673137, [])
(1696549691, [])
(514021382, [])
(1245195368, [])
(1292379241, [])
(1002183790, [])
(1231604051, [])
(518837367, [])
(1431323140, [])
(552998034, [])
(1258537770, [])
(213093155, [])
(1975460849, [])
(1341276954, [])
(784802953, [])
(2094003179, [])
(1505383192, [])
(1656687756, [])
(2024782818, [])
(825843615, [])
(2039021366

KeyboardInterrupt: 

In that case, only one of the rows with the same `id` entry need to be kept.

In [263]:
df.drop_duplicates(subset = "id", keep = "first", inplace = True)

In [264]:
df.shape

(168979, 13)

Now we will take a look in to `location` and try to extract some meaningful information.

Fist we will drop NaN rows and then convert the `json-str` into a dictonary to extract keys and values.

In [265]:
df.location.isnull().sum()

213

In [266]:
df.dropna(axis = 0, subset = ["location"], inplace = True)

In [267]:
df.location =[json.loads(x) for x in df.location]

In [268]:
df["location_type"] = [df.location.iloc[x]["type"] for x in range(0,len(df.location))]
df["location_city"] = [df.location.iloc[x]["short_name"] for x in range(0,len(df.location))]
df["location_state"] = [df.location.iloc[x]["state"] for x in range(0,len(df.location))]

In [269]:
df.location_type.value_counts()

Town             156447
County             6384
Suburb             4295
LocalAdmin          986
Zip                 409
Island              209
Country              17
Miscellaneous        15
Estate                4
Name: location_type, dtype: int64

Remaining questions:
- What to do with location? Extract cities and try to assign some score to each? To test this, extract a few big cities and check for correlation with target.

In [270]:
df.shape

(168766, 16)